In [1]:
! pwd

/Users/sarahwooller/Sussex/CRUK/vision/CRUK_datahub_landing_page/python_helper_code_and_jsons


In [2]:
import os
import json

In [3]:
path = "../src/utils/new_filter_data.js"

In [4]:
with open(path) as f:
    text = f.read().split("\n")

In [5]:
dictionary = eval(text[1][:-1])

In [6]:
with open("../src/utils/fd.json","w") as f:
    json.dump(dictionary, f)

In [7]:
dictionary["0_1"]

{'id': '0_1',
 'label': 'accessType',
 'category': 'filters',
 'primaryGroup': 'access-type',
 'count': '0',
 'children': {'0_1_0': {'id': '0_1_0',
   'label': 'Access restricted at present',
   'category': 'accessType',
   'primaryGroup': 'access-type',
   'count': '0'},
  '0_1_1': {'id': '0_1_1',
   'label': 'Closed to access',
   'category': 'accessType',
   'primaryGroup': 'access-type',
   'count': '0'},
  '0_1_2': {'id': '0_1_2',
   'label': 'Open in response to specific calls',
   'category': 'accessType',
   'primaryGroup': 'access-type',
   'count': '0'},
  '0_1_3': {'id': '0_1_3',
   'label': 'Open only through collaboration',
   'category': 'accessType',
   'primaryGroup': 'access-type',
   'count': '0'},
  '0_1_4': {'id': '0_1_4',
   'label': 'Open to applicants',
   'category': 'accessType',
   'primaryGroup': 'access-type',
   'count': '0'}}}

In [12]:
from typing import Dict, Any, List, Optional

class TreeNode:
    """
    Represents a single node in the hierarchical tree structure.
    """
    def __init__(
        self,
        id: str,
        label: str,
        category: str,
        primaryGroup: str,
        count: str,
        description: str,
        children: Optional[List['TreeNode']] = None,
    ):
        """
        Initializes a new tree node with data attributes.

        Args:
            id (str): The unique identifier for the node.
            label (str): The human-readable label or name.
            category (str): The category of the node.
            primary_group (str): The primary group classification.
            count (str): A count associated with the node.
            children (Optional[List['TreeNode']]): A list of child TreeNode objects.
        """
        self.id = id
        self.label = label
        self.category = category
        self.primaryGroup = primaryGroup
        self.description = description
        self.count = count
        self.children: List['TreeNode'] = children if children is not None else []

    def __repr__(self):
        """
        Provides a string representation for debugging.
        """
        return f"TreeNode(id='{self.id}', label='{self.label}', children={len(self.children)})"

    def print_tree(self, level=0):
        """
        Recursively prints the structure of the tree.
        """
        indent = "  " * level
        print(f"{indent}- {self.label} (ID: {self.id})")
        for child in self.children:
            child.print_tree(level + 1)

In [20]:
def build_dict(tree: TreeNode) -> dict:
    "Recursively builds the new dictionary from the tree"

    dictionary = {}
    for label in ["id", "label", "category", "primaryGroup", "count"]:
        dictionary[label] = getattr(tree, label)
    if tree.children:
        dictionary["children"] = dict([(child.id, build_dict(child)) for child in tree.children])
    return dictionary
    

In [14]:
def build_tree(data: Dict[str, Any]) -> TreeNode:
    """
    Recursively builds the TreeNode structure from the raw dictionary data.

    Args:
        data (Dict[str, Any]): The raw dictionary data for a single node.

    Returns:
        TreeNode: The resulting object-oriented tree node.
    """
    # Extract core attributes
    node_id = data.get('id', '')
    label = data.get('label', '')
    category = data.get('category', '')
    primaryGroup = 'cancer-type'
    count = data.get('count', '')
    description = data.get('description', '')

    # Initialize the current node
    current_node = TreeNode(
        id=node_id,
        label=label,
        category=category,
        primaryGroup=primaryGroup,
        count=count,
        description=description
    )

    # Check for children and recurse
    raw_children = data.get('children', {})
    if raw_children and isinstance(raw_children, dict):
        # Iterate over the values (the child dictionaries)
        for child_dict in raw_children.values():
            # Recursively build the child node
            child_node = build_tree(child_dict)
            current_node.children.append(child_node)

    return current_node

In [15]:
tree = build_tree(dictionary['0_0'])

In [16]:
tcga = tree.children.pop()

In [17]:
for child in tcga.children:
    child.label = f"{child.label} ({child.description})"

In [18]:
tree.children.append(tcga)

In [21]:
new_dictionary = build_dict(tree)

In [22]:
dictionary['0_0'] = new_dictionary

In [23]:
path = "../src/utils/longer_filter_data.js"

In [24]:
with open(path, "w") as f:
    f.write(text[0])
    f.write("\n")

In [25]:
with open(path, "a") as f:
    json.dump(dictionary, f)

In [26]:
with open(path, "a") as f:
    f.write(";\n")
    f.write(text[2])

In [27]:
with open("../src/utils/lfd.json","w") as f:
    json.dump(dictionary, f)